In [92]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
import seaborn as sns
sns.set(color_codes=True)
import geopandas as gp
from scipy import stats, integrate

ImportError: No module named 'series'

In [46]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOWavailable.csv', encoding='utf-8')
herepath.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW
0,0,998969850F,2014-01-01 13:55,167,186,61.0,0.0,10,Wednesday
1,1,998969850F,2014-01-02 11:05,133,186,60.0,0.0,10,Thursday
2,2,998969850F,2014-01-02 13:20,160,186,61.0,0.0,10,Thursday
3,3,998969850F,2014-01-02 14:30,174,186,52.0,0.0,10,Thursday
4,4,998969850F,2014-01-03 09:05,109,186,57.0,0.0,10,Friday


In [20]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [21]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [22]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [23]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe(w,x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        link = v[v['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        link = x[x['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [166]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [167]:
data = timeframe('998969850F',herepath,0,288,True,Weekend)
data.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW,DATE
10,10.0,998969850F,2014-01-04 11:20,136.0,186.0,46.0,0.0,10.0,Saturday,2014-01-04
11,11.0,998969850F,2014-01-04 14:30,174.0,186.0,43.0,0.0,10.0,Saturday,2014-01-04
12,12.0,998969850F,2014-01-04 17:30,210.0,186.0,54.0,0.0,10.0,Saturday,2014-01-04
13,13.0,998969850F,2014-01-05 17:10,206.0,186.0,57.0,0.0,10.0,Sunday,2014-01-05
50,50.0,998969850F,2014-01-11 14:15,171.0,186.0,62.0,0.0,10.0,Saturday,2014-01-11


In [168]:
herepath.quantile(0.05,interpolation = 'nearest')

Unnamed: 0    67393.0
EPOCH_5MIN       69.0
LENGTH           14.0
MEAN             23.0
STDDEV            0.0
CONFIDENCE       10.0
Name: 0.05, dtype: float64

In [170]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstats(x):
    print('Mean is ', np.mean(x['MEAN']), 'KMH')
    var = ((x['MEAN'].std(ddof=1))**2+(((x['STDDEV'])**2).sum()/len(x)))
    print('Standard Deviation is ', np.sqrt(var),'KMH')
    print('85th percentile speed is ', x['MEAN'].quantile(0.85,interpolation = 'lower' ), 'KMH')
    print('5th percentile speed is ', x['MEAN'].quantile(0.05,interpolation = 'lower'), 'KMH')
    
    ax = sns.distplot(x['MEAN'], kde=False, bins=70)
    ax.figure.set_size_inches(8,6)
    ax.set_xlim(0,100)
    return ax

In [171]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsGB(x):
    y = pd.DataFrame(columns=['Mean','StDev','FFS','FiveSpd'])
    y['Mean'] = (x['MEAN']).groupby(x['LINK_DIR']).mean()
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.std(ddof=1))**2+(Var2.sum()/len(x))
    
    y['StDev'] = np.sqrt(Var)
    
    y['FFS'] = x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower')
    
    y['FiveSpd'] = x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower')
    
    return y

In [172]:
#For all days and all time periods
bylink = HEREstatsGB(herepath)
bylink.head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
107030432T,62.051514,16.809469,78.0,31.0
107030439T,56.572330,15.981771,72.0,28.0
107030440T,56.113229,16.582816,73.0,27.5
107030444F,58.232521,18.041906,76.0,26.0
107030445F,58.912403,18.107840,77.0,27.0


In [176]:
#for weekdays
Wkd = timeframe2(herepath,0,288,True,Weekday)
HEREstatsGB(Wkd).head()

MemoryError: 

In [175]:
#for weekends
Wend = timeframe2(herepath,0,288,True,Weekend)
HEREstatsGB(Wend)

MemoryError: 

In [ ]:
#Weekday Between 6AM and 8PM
Wdtime = timeframe2(herepath,72,240,True,Weekday)
HEREstatsGB(Wdtime)

In [ ]:
#Stats between 6AM and 9AM
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
HEREstatsGB(WdtimeAM)

In [ ]:
#Stats between 9AM and 3PM
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
HEREstatsGB(WdtimeMD)

In [ ]:
#Stats between 3PM and 6PM
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
HEREstatsGB(WdtimePM)